In [1]:
import numpy as np
from copy import deepcopy

In [2]:
class Input:
  def __init__(self, File):
    inputs=File.read().split()
    inputs=[int(i) for i in inputs]
    self.v=inputs[0]
    self.u=inputs[1]
    
    self.a_matrix=np.zeros((self.v,self.u)) #Initializing adjacency matrix
    
    self.v_list=[[] for i in range(self.v)] #Initializing adjacency lists
    self.u_list=[[] for i in range(self.u)]
    
    self.s_v_list=np.zeros((self.v,self.v)) #Initializing second order adjacency lists
    self.s_u_list=np.zeros((self.u,self.u))
    
    for i,x in enumerate(inputs[2:]):
        if(x==0):
            continue
        current_v=int(i/self.u)
        current_u=i%self.u
        self.v_list[current_v].append(current_u)
        self.u_list[current_u].append(current_v)
        self.a_matrix[current_v][current_u]=1
    #Getting Second neighboorhood through matrix.
    v_sec=np.matmul(self.a_matrix,self.a_matrix.T)
    for i in range(self.v):
        for j in range(self.v):
            if(v_sec[i][j]>0):
                self.s_v_list[i][j]=1
                self.s_v_list[j][i]=1
    u_sec=np.matmul(self.a_matrix.T,self.a_matrix)
    for i in range(self.u):
        for j in range(self.u):
            if(u_sec[i][j]>0):
                self.s_u_list[i][j]=1
                self.s_u_list[j][i]=1
    print(self.s_v_list)
    #for a,b in enumerate(self.v_list):
    #    for c in b:
    #        for d in self.u_list[c]:
    #            if(self.s_v_list[a][d]==0):
    #                self.s_v_list[a][d]=1
    #                self.s_v_list[d][a]=1
    #for a,b in enumerate(self.u_list):
    #    for c in b:
    #        for d in self.v_list[c]:
    #            if(self.s_u_list[a][d]==0):
    #                self.s_u_list[a][d]=1
    #                self.s_u_list[d][a]=1
    self.a_matrix=(self.a_matrix-0.5)*2
    
class Bicluster:
    def __init__(self, v_array,u_array):
        self.v_list=v_array
        self.u_list=u_array
        
class Solution:
    
    def __init__(self,entrada,clusters):
        self.Input=entrada
        if(clusters):
            self.n_clusters=0
            self.a_matrix=np.zeros((entrada.v,entrada.u))
            self.v_clusters=clusters
            for a in self.v_clusters:
                for b in a.v_list:
                    for c in a.u_list:
                        self.a_matrix[b][c]=1
            self.edition_matrix()
        else:
            self.n_clusters=0
            self.a_matrix=np.zeros((entrada.v,entrada.u))
            self.v_clusters=self.greedy()
            for a in self.v_clusters:
                for b in a.v_list:
                    for c in a.u_list:
                        self.a_matrix[b][c]=1
            self.edition_matrix()
        
        
    def evaluate(self):
        npm1=self.Input.a_matrix
        npm2=self.a_matrix
        return abs(npm1-npm2).sum()
        #return abs(np.matrix((npm1==npm2)-1).sum())
        
    def recomendations(self,e_a_matrix):
        npm1=np.matrix(e_a_matrix)
        npm2=np.matrix(self.a_matrix)
        rec=(npm1==npm2-1)
        rec_list=[]
        for i,x in enumerate(rec):
            for j,y in enumerate(x.flat):
                if(y):
                    rec_list.append([i,j])
        return rec_list
    
    def greedy(self):
        
        g_v_list=self.Input.v_list.copy()
        g_u_list=self.Input.u_list.copy()
        links=[]
        current_value=0
        current_id=[0,0]
        biclusters=[]
        for i,x in enumerate(g_v_list):
            for j,y in enumerate(g_u_list):
                current_id=[i,j]
                current_value=self.Input.a_matrix[i][j]
                v_links=len(x)
                u_links=len(y)
                for k,z in enumerate(self.Input.s_u_list[j]): # i Part of the in and diff formula
                    if(k==j):
                        continue
                    if(k in x):
                        if(z==0):
                            z=-1
                        else:
                            v_links-=1
                        current_value+=z

                for k,z in enumerate(self.Input.s_v_list[i]): # j part of the in and diff formula
                    if(k==i):
                        continue
                    if(k in y):
                        if(z==0):
                            z=-1
                        else:
                            u_links-=1
                        current_value+=z
                if(u_links<0 or v_links<0):
                    print("Error in number of links")
                    return "Error in number of links"
                current_value-=u_links+v_links
                links.append([current_id[0],current_id[1],current_value])
        v_exclusions=np.zeros(self.Input.v)
        u_exclusions=np.zeros(self.Input.u)
        while(links):
            links.sort(key=lambda x: x[2],reverse=True)
            
            v=links[0][0]
            u=links[0][1]
            
            nv=g_v_list[v]
            nu=g_u_list[u]
            
        
            if(self.Input.a_matrix[v][u]):
                nv.append(u)
                nu.append(v)
            aux_nu=[]
            for x in nu:
                if(not v_exclusions[x]):
                    aux_nu.append(x)
                    v_exclusions[x]=1
                
            aux_nv=[]
            for y in nv:
                if(not u_exclusions[y]):
                    aux_nv.append(y)
                    u_exclusions[y]=1  
                                             
            biclusters.append(Bicluster(aux_nu,aux_nv))
            self.n_clusters+=1
            
            l=len(links)
            newlinks=[]
            for c in range(l):
                if(v_exclusions[links[c][0]] or u_exclusions[links[c][1]]):
                    continue
                else:
                    newlinks.append(links[c])
            links=newlinks
            
        return biclusters
        
    def edition_matrix(self):
        a_matrix=self.Input.a_matrix.copy()
        biclusters=self.v_clusters
        self.m_v_edit=np.zeros((self.Input.v,len(biclusters)))
        self.m_u_edit=np.zeros((self.Input.u,len(biclusters)))
        for i in range(self.Input.v):
            for j in range(len(biclusters)):
                if(i in biclusters[j].v_list):
                    for x in biclusters[j].u_list:
                        self.m_v_edit[i][j]+=a_matrix[i][x]
                else:
                    for x in biclusters[j].u_list:
                        self.m_v_edit[i][j]-=a_matrix[i][x]
        for i in range(self.Input.u):
            for j in range(len(biclusters)):
                if(i in biclusters[j].u_list):
                    for x in biclusters[j].v_list:
                        self.m_u_edit[i][j]+=a_matrix[x][i]
                else:
                    for x in biclusters[j].v_list:
                        self.m_u_edit[i][j]-=a_matrix[x][i]
        return self.m_v_edit,self.m_u_edit
    
    def select_vertex(self): #Selects the best vertex to move from a cluster to another
        m_v_edit,m_u_edit=self.m_v_edit,self.m_u_edit
        biclusters=self.v_clusters.copy()
        change_id=[-1,-1,-1]
        change_value=0
        change_flag=0 #0 for v 1 for u
        current_value=0
        for x in range(self.Input.v):
            for k,y in enumerate(biclusters):
                if(x in y.v_list):
                    i=k
                    break
                else:
                    i=-1
            for j in range(len(biclusters)):
                if(i==j):
                    continue
                if(change_id==[-1,-1,-1]):
                    change_id=[x,i,j]
                    change_value=m_v_edit[x][i]+m_v_edit[x][j]
                else:
                    current_value=m_v_edit[x][i]+m_v_edit[x][j]
                    if(current_value<change_value):
                        change_id=[x,i,j]
                        change_value=current_value
        for x in range(self.Input.u):
            for k,y in enumerate(biclusters):
                if(x in y.u_list):
                    i=k
                    break
                else:
                    i=-1
            for j in range(len(biclusters)):
                if(i==j):
                    continue
                if(change_id==[-1,-1,-1]):
                    change_id=[x,i,j]
                    change_value=m_u_edit[x][i]+m_u_edit[x][j]
                    change_flag=1
                else:
                    current_value=m_u_edit[x][i]+m_u_edit[x][j]
                    if(current_value<change_value):
                        change_id=[x,i,j]
                        change_value=current_value
                        change_flag=1
        return change_id,change_flag
    
    def cluster_union(self):
        if(self.n_clusters==1):
            return "Apenas um cluster"
        best_sol=deepcopy(self)
        best_value=best_sol.evaluate()
        found=0
        for i,x in enumerate(self.v_clusters):
            for j,y in enumerate(self.v_clusters):
                current_sol=deepcopy(self)
                biclusters=current_sol.v_clusters
                for x in biclusters[i].v_list:
                    biclusters[i].v_list.remove(x)
                    biclusters[j].v_list.append(x)            
                for x in biclusters[i].u_list:
                    biclusters[i].u_list.remove(x)
                    biclusters[j].u_list.append(x) 
                if(biclusters[i].v_list==[] and biclusters[i].u_list==[]):
                    biclusters.pop(i)
                    current_sol.n_clusters-=1
                else:
                    return "Error"
                if(current_sol.evaluate()<best_value):
                    best_sol=deepcopy(current_sol)
                    best_value=best_sol.evaluate()
                    found=1
        if(found==1):
            return best_sol
        else:
            return 0
        
    def cluster_break(self):
        best_sol=deepcopy(self)
        best_value=best_sol.evaluate()
        found=0
        for i,x in enumerate(self.v_clusters):
            current_sol=deepcopy(self)
            biclusters=current_sol.v_clusters
            for x in biclusters[i].v_list:
                biclusters[i].v_list.remove(x)
                biclusters[j].v_list.append(x)            
            for x in biclusters[i].u_list:
                biclusters[i].u_list.remove(x)
                biclusters[j].u_list.append(x) 
            if(biclusters[i].v_list==[] and biclusters[i].u_list==[]):
                biclusters.pop(i)
                current_sol.n_clusters-=1
            else:
                return "Error"
            if(current_sol.evaluate()<best_value):
                best_sol=deepcopy(current_sol)
                best_value=best_sol.evaluate()
                found=1
        if(found==1):
            return best_sol
        else:
            return 0
                    
    def move_vertex(self,change_arg):
        change_id,change_flag=change_arg #Change_id [vertice,bicluster saida, bicluster entrada]
        new_sol=deepcopy(self)
        biclusters=new_sol.v_clusters
        x,i,j=change_id
        if(change_flag):# U partition
            for v in biclusters[i].v_list:
                new_sol.a_matrix[v][x]=0
            for v in biclusters[j].v_list:
                new_sol.a_matrix[v][x]=1
            biclusters[i].u_list.remove(x)
            biclusters[j].u_list.append(x)
            for v in range(new_sol.Input.v):
                if(v in biclusters[i].v_list):
                    new_sol.m_v_edit[v][i]-=new_sol.Input.a_matrix[v][x]
                    new_sol.m_v_edit[v][j]-=new_sol.Input.a_matrix[v][x]
                else:
                    new_sol.m_u_edit[v][i]+=new_sol.Input.a_matrix[v][x]
                    new_sol.m_u_edit[v][j]+=new_sol.Input.a_matrix[v][x]
        else: # V partition
            for u in biclusters[i].u_list:
                new_sol.a_matrix[x][u]=0
            for u in biclusters[j].u_list:
                new_sol.a_matrix[x][u]=1
            biclusters[i].v_list.remove(x)
            biclusters[j].v_list.append(x)
            new_sol.m_v_edit[x][i]=-new_sol.m_v_edit[x][i]
            new_sol.m_v_edit[x][j]=-new_sol.m_v_edit[x][j]
            for u in range(new_sol.Input.u):
                if(u in biclusters[i].u_list):
                    new_sol.m_u_edit[u][i]-=new_sol.Input.a_matrix[x][u]
                    new_sol.m_u_edit[u][j]-=new_sol.Input.a_matrix[x][u]
                else:
                    new_sol.m_u_edit[u][i]+=new_sol.Input.a_matrix[x][u]
                    new_sol.m_u_edit[u][j]+=new_sol.Input.a_matrix[x][u]
                
        if(biclusters[i].v_list==[] and biclusters[i].u_list==[]):
            biclusters.pop(i)
            new_sol.n_clusters-=1
        return new_sol

In [18]:
f = open("V_5_U_7_D_0.5_S_1.dat",'r')
entrada=Input(f)
print(entrada.a_matrix[3][6])
solucao=Solution(entrada,0)
new_sol=solucao.move_vertex(solucao.select_vertex())
print(solucao.a_matrix,new_sol.a_matrix,sep='\n\n')

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 0. 1.]]
-1.0
[[1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0.]]

[[1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0.]]
